In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

def extract_canonical_urls(data, urls=[]):
    if isinstance(data, dict):
        for key, value in data.items():
            if key == "canonical_url":
                urls.append(value)
            elif isinstance(value, (dict, list)):
                extract_canonical_urls(value, urls)
    elif isinstance(data, list):
        for item in data:
            extract_canonical_urls(item, urls)

    return urls

def sanitize_headers(headers):
    safe_headers = {}
    for k, v in headers.items():
        try:
            v.encode("latin-1")
            safe_headers[k] = v
        except UnicodeEncodeError:
            # Skip or clean this header value
            safe_headers[k] = v.encode("ascii", "ignore").decode("ascii")
    return safe_headers


num_queries=200
all_links=[]

for i in range(1, num_queries):
    if i%10 == 0:
        print(f"First {100*i} links... Length: {len(all_links)}")

    url = "https://www.semana.com/pf/api/v3/content/fetch/content-apir"
    params = {
        "query": '{"feedOffset":' + str(100*i) + ',"feedSize":100,"includeSections":"politica","sourceInclude":"canonical_url,_id,promo_items,headlines.basic,subheadlines.basic,description.basic,subtype,publish_date,taxonomy,copyright"}',
        "d": "7705",
        "mxId": "00000000",
        "_website": "semana"
    }
    
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-US,en;q=0.5",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Cookie": "_ga=GA1.1.1191394108.1661543491; G_ENABLED_IDPS=google; _cb=Dsqyd2C1x1TI8yoHn; _chartbeat2=.1661543509533.1733159429346.0000000000000001.BjmQrPCWqbIOBQpCxuCNvJsnCsOTO-.1; cto_bundle=z4kmfV9xZVY5UlZOQ2VZdXR2JTJGcEUyNVRlWnlLZEY3YkZlWDNZdlN0bVQ5RFJCUGxnSmE3bGRZQ2tPMVk4MXY3WlJmRjJ4MW5CdGp4Wml3JTJCcFFkYWlwNkhYRjZvaHVoRFdPaDdqU0xpSVRPZHB1aHpzdW5ybHpGUkk2JTJCV2VLcXFmaFVOYW1JM3R5WUpOcVNPbWMzS0x1Y1ZFR2lXbEhBbiUyRjNNdVd2eXlEc2RXSjVsV0dkdHJXcjdydDhJY0xFTlNmQ3dNeg; _ga_WM4PJWMJZ5=GS2.1.s1748516709$o18$g1$t1748516736$j3…l=0&tt=0; panoramaId_expiry=1748633734968; panoramaId=b4309327c7bb910a48a7f949f96e185ca02cf9e45c2db100701d9c774eba9af4; panoramaIdType=panoDevice; AKA_A2=A; FCCDCF=%5Bnull%2Cnull%2Cnull%2Cnull%2Cnull%2Cnull%2C%5B%5B13%2C%22%5B%5C%22DBABL~BVQqAAAAAg%5C%22%2C%5B%5B7%2C%5B1748516712%2C296424000%5D%5D%5D%5D%22%5D%5D%5D; FCNEC=%5B%5B%22AKsRol8oGc5TIm-5Lgjq3eFr8TwLysWlO44hKZyWyOaMCICTkzUDM_M6p3-5CZ6wRr6tZ4vbaNGA034bN1_K-kYUgch1uJO_j2V3dxm013j_56hCqjbECGPEJd57bDmy96f9hzwjN26UExwsHeCdgTwX5irz46zUnw%3D%3D%22%5D%5D",
        "Host": "www.semana.com",
        "If-Modified-Since": "1748516231192",
        "Pragma": "no-cache",
        "Priority": "u=4",
        "Referer": "https://www.semana.com/politica/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "TE": "trailers"
    }
    
    response = requests.get(url, headers=sanitize_headers(headers), params=params)
    links = extract_canonical_urls(json.loads(response.text))
    all_links = all_links + links



ChunkedEncodingError: ('Connection broken: IncompleteRead(15 bytes read, 1 more expected)', IncompleteRead(15 bytes read, 1 more expected))

In [2]:
requests.get(url, headers=sanitize_headers(headers), params=params)

ChunkedEncodingError: ('Connection broken: IncompleteRead(15 bytes read, 1 more expected)', IncompleteRead(15 bytes read, 1 more expected))

In [46]:

url = "https://www.semana.com/"
#df = pd.DataFrame()

# Perform GET requests for each filtered hyperlink
for i, link in enumerate(links[4413:]):
        
    full_url = url.rstrip('/') + link  # Construct the full URL by combining the base URL with the relative link
    link_response = requests.get(full_url)
#    print(f"Status code for {full_url}: {link_response.status_code}")
    link_soup = BeautifulSoup(link_response.text, 'html.parser')
    
    soup = BeautifulSoup(link_response.content, "html.parser")

    # Find all the script tags with type="application/ld+json"
    script_tags = soup.find_all('script', {'type': 'application/ld+json'})

    for script_tag in script_tags:
        try:
            # Load the JSON data from the script tag
            json_data = json.loads(script_tag.string)

            # Check if the desired keys are present
            if 'datePublished' in json_data and 'headline' in json_data and 'description' in json_data:
                # Extract the 'datePublished', 'headline', and 'description' values
                date_published = json_data['datePublished']
                headline = json_data['headline']
                description = json_data['description']
                dateModified = json_data["dateModified"]
                dateline = json_data["dateline"]
                alternativeHeadline = json_data["alternativeHeadline"]
                keywords = json_data["keywords"]
                articleSection = json_data["articleSection"]
                articleBody = json_data["articleBody"]

                break
                
        except json.JSONDecodeError:
            # Skip this script tag if there's an error decoding JSON
            date_published = 'NA'
            headline = 'NA'
            description = 'NA'
            articleBody = 'NA'
            dateModified = 'NA'
            dateline = 'NA'
            alternativeHeadline = 'NA'
            keywords = 'NA'
            articleSection = 'NA'
            pass
    
    df = pd.concat([df, pd.DataFrame({
                                      'articleBody': [articleBody],
                                      'date_published': [date_published],
                                      'headline': [headline],
                                      'description': [description],
                                      'dateModified': [dateModified],
                                      'dateline': [dateline],
                                      'alternativeHeadline': [alternativeHeadline],
                                      'keywords': [keywords],
                                      'articleSection': [articleSection]
                                     })])
    if i%500 == 0:
        print(f"Extracted content of first {i} links... Date: {date_published}")
        df.to_csv("datos_politica.csv", index=False, encoding='utf-8-sig')
    

Extracted content of first 0 links... Date: 2024-09-26T15:39:44-05:00
Extracted content of first 500 links... Date: 2024-09-03T09:58:42-05:00
Extracted content of first 1000 links... Date: 2024-08-08T14:50:48-05:00
Extracted content of first 1500 links... Date: 2024-07-18T20:07:35-05:00
Extracted content of first 2000 links... Date: NA
Extracted content of first 2500 links... Date: 2024-05-29T20:47:23-05:00
Extracted content of first 3000 links... Date: 2024-05-10T11:26:21-05:00
Extracted content of first 3500 links... Date: 2024-04-21T14:15:58-05:00


KeyError: 'dateline'

In [48]:
i

3974